In [1]:
%%file Convolution2D.py
import torch
import torch.nn.functional as F
import numpy as np

def cv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'):
    def convolution2d(input_m):
        if bias:
            value_b = torch.rand(out_channels)
        else:
            value_b = torch.zeros(out_channels)

        # Проверяем правила для свертки с группами
        assert in_channels % groups == 0
        assert out_channels % groups == 0

        if padding_mode == 'zeros':
            input_m = F.pad(input_m, (padding, padding, padding, padding), mode='constant', value=0)
        elif padding_mode == 'reflect':
            input_m = F.pad(input_m, (padding, padding, padding, padding), mode='reflect')
        elif padding_mode == 'replicate':
            input_m = F.pad(input_m, (padding, padding, padding, padding), mode='replicate')
        elif padding_mode == 'circular':
            input_m = circular_pad(input_m, padding)
        else:
            raise ValueError("Unsupported padding_mode")

        if type(kernel_size) == tuple:
            filter = torch.rand(out_channels, in_channels // groups, kernel_size[0], kernel_size[1])
        elif type(kernel_size) == int:
            filter = torch.rand(out_channels, in_channels // groups, kernel_size, kernel_size)
        else:
            raise ValueError("Unsupported kernel_size type")

        out_tensor = []
        for l in range(out_channels):
            f = np.array([])
            for i in range (0, input_m.shape[1] - ((filter.shape[2]-1) * dilation + 1) + 1, stride):
                for j in range (0, input_m.shape[2] - ((filter.shape[3]-1) * dilation + 1) + 1, stride):
                    s = 0
                    for c in range (in_channels//groups):
                        if groups > 1:
                            val = input_m[l * (in_channels//groups) + c][i:i + (filter.shape[2]-1) * dilation + 1:dilation, j:j + (filter.shape[3]-1) * dilation + 1:dilation]
                        else:
                            val = input_m[c][i:i + (filter.shape[2]-1) * dilation + 1:dilation, j:j + (filter.shape[3] - 1) * dilation + 1:dilation]
                        mini_sum = (val * filter[l][c]).sum()
                        s = s + mini_sum
                    f = np.append(f, float(s + value_b[l]))
            out_tensor.append(torch.tensor(f, dtype=torch.float).view(1, 1, -1))
        return np.array(out_tensor), torch.tensor(np.array(filter)), torch.tensor(np.array(value_b))
    return convolution2d


Overwriting Convolution2D.py


In [2]:
%%file test_convolution2d.py
import torch
import pytest
from Convolution2D import cv2d
import torch.nn as nn

def test_cv2d_1():
    tensor = torch.rand(8, 5, 6)

    Convolution2D = cv2d(in_channels=8, out_channels=4, kernel_size=3, stride=1, padding=0, dilation=1, groups=4, bias=True, padding_mode='zeros')
    result, kernel_size, bias = Convolution2D(tensor)
    torchFunction = nn.Conv2d(in_channels=8, out_channels=4, kernel_size=3, stride=1, padding=0, dilation=1, groups=4, bias=True, padding_mode='zeros')
    torchFunction.weight.data = torch.tensor(kernel_size)
    torchFunction.bias.data = torch.tensor(bias)

def test_cv2d_2():
    tensor = torch.rand(4, 5, 5)

    Convolution2D = cv2d(in_channels=4, out_channels=2, kernel_size=3, stride=2, padding=2, dilation=1, groups=2, bias=True, padding_mode='reflect')
    result, kernel_size, bias = Convolution2D(tensor)
    torchFunction = nn.Conv2d(in_channels=4, out_channels=2, kernel_size=3, stride=2, padding=2, dilation=1, groups=2, bias=True, padding_mode='reflect')
    torchFunction.weight.data = torch.tensor(kernel_size)
    torchFunction.bias.data = torch.tensor(bias)

def test_cv2d_3():
    tensor = torch.rand(2, 2, 2)

    Convolution2D = cv2d(in_channels=2, out_channels=2, kernel_size=2, stride=2, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
    result, kernel_size, bias = Convolution2D(tensor)
    torchFunction = nn.Conv2d(in_channels=2, out_channels=2, kernel_size=2, stride=2, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
    torchFunction.weight.data = torch.tensor(kernel_size)
    torchFunction.bias.data = torch.tensor(bias)


Overwriting test_convolution2d.py


In [3]:
!pytest 

============================= test session starts =============================
platform win32 -- Python 3.9.16, pytest-7.1.2, pluggy-1.0.0
rootdir: d:\SMZ\lab1
plugins: anyio-3.5.0
collected 3 items

test_convolution2d.py ...                                                [100%]

============================== warnings summary ===============================
test_convolution2d.py::test_cv2d_1
test_convolution2d.py::test_cv2d_2
test_convolution2d.py::test_cv2d_3
  d:\SMZ\lab1\Convolution2D.py:49: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
    return np.array(out_tensor), torch.tenso

In [4]:
import numpy as np

def convolution_2d(input_tensor, weight, bias, stride=1, padding=0):
    # Получаем размеры входного тензора и ядра
    batch_size, in_channels, in_height, in_width = input_tensor.shape
    out_channels, _, kernel_size, _ = weight.shape

    # Рассчитываем размеры выходного тензора
    out_height = (in_height + 2 * padding - kernel_size) // stride + 1
    out_width = (in_width + 2 * padding - kernel_size) // stride + 1

    # Создаем выходной тензор
    output_tensor = np.zeros((batch_size, out_channels, out_height, out_width))

    # Применяем свертку
    for b in range(batch_size):
        for oc in range(out_channels):
            for oh in range(0, out_height * stride, stride):
                for ow in range(0, out_width * stride, stride):
                    # Выделяем кусок входного тензора
                    input_slice = input_tensor[b, :, oh:oh+kernel_size, ow:ow+kernel_size]

                    # Применяем свертку (поэлементное умножение и суммирование)
                    output_tensor[b, oc, oh//stride, ow//stride] = np.sum(input_slice * weight[oc]) + bias[oc]

    return output_tensor


In [13]:
def test_convolution_2d():
    # Создаем входной тензор, веса и смещение
    input_tensor = np.random.rand(2, 3, 4, 4)
    weight = np.random.rand(5, 3, 3, 3)
    bias = np.random.rand(5)

    # Вызываем нашу функцию свертки
    custom_output = convolution_2d(input_tensor, weight, bias)

    # Используем встроенную функцию PyTorch для свертки
    import torch
    import torch.nn as nn

    torch_conv = nn.Conv2d(3, 5, kernel_size=3)
    torch_output = torch_conv(torch.tensor(input_tensor, dtype=torch.float32))
    print("Custom Output:")
    print(custom_output)
    print("\nPyTorch Output:")
    print(torch_output.detach().numpy())

    print("\nDifference:")
    print(custom_output - torch_output.detach().numpy())

    # Проверяем, совпадают ли результаты
    # np.testing.assert_allclose(custom_output, torch_output.detach().numpy(), rtol=1e-5)

# Запускаем тесты
test_convolution_2d()


Custom Output:
[[[[6.20261047 6.8446192 ]
   [5.75773456 6.86908592]]

  [[6.55674849 6.82567574]
   [6.60225017 6.1037361 ]]

  [[6.02688642 6.26373563]
   [6.05833308 6.05528529]]

  [[5.87503785 5.14720024]
   [4.97920085 5.27788051]]

  [[5.9773733  6.21228235]
   [5.77192805 5.66118583]]]


 [[[9.04392416 7.64726451]
   [9.86783607 7.07144742]]

  [[8.62346548 9.32910253]
   [9.49971242 8.14498258]]

  [[8.26695763 8.32120175]
   [7.92308355 7.44420464]]

  [[7.57249632 6.55402827]
   [7.00368381 6.84683706]]

  [[8.62039515 8.86813249]
   [8.63951339 7.59392805]]]]

PyTorch Output:
[[[[ 0.37422693  0.6152117 ]
   [ 0.6018531   0.6573878 ]]

  [[ 0.248276   -0.07862993]
   [ 0.2556469   0.02925629]]

  [[ 0.5145069   0.09180786]
   [ 0.47800478  0.21590856]]

  [[-0.12584819 -0.026763  ]
   [ 0.0746723  -0.1294945 ]]

  [[ 0.11321617  0.26693416]
   [ 0.04741885  0.56894374]]]


 [[[ 0.68607926  0.663958  ]
   [ 0.6664135   0.64351314]]

  [[ 0.47675046  0.02494059]
   [ 0.2716239

In [1]:
from torch import nn
import numpy as np
import matplotlib.pyplot as plt

# Transforms
_transform = A.Compose([
    A.Resize(height = 512, width=512), 
    ToTensorV2(), 
])


trans_image = _transform(image=np.array(image))
outputs = model(trans_image['image'].float().unsqueeze(0))
logits = outputs.logits.cpu()
print(logits.shape)


# First, rescale logits to original image size
upsampled_logits = nn.functional.interpolate(logits,
                size=image.size[::-1], # (height, width)
                mode='bilinear',
                align_corners=False)


seg = upsampled_logits.argmax(dim=1)[0]
color_seg = np.zeros((seg.shape[0], seg.shape[1], 3), dtype=np.uint8) # height, width, 3
palette = np.array([[0, 0, 0],[255, 255, 255]])
for label, color in enumerate(palette):
    color_seg[seg == label, :] = color
# Convert to BGR
color_seg = color_seg[..., ::-1]


NameError: name 'A' is not defined